Imports

In [ ]:
#connecting to google drive
from google.colab import drive
drive.mount('/content/drive')

#imports
!pip install contractions
import numpy as np
import os
import contractions
import pandas as pd
import spacy
import nltk
import string
import re
from collections import Counter
import matplotlib.pyplot as plt
import unicodedata
import locale
from datetime import datetime

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 85.1 MB/s eta 0:00:00
   ━━━━━━━

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

Load of the data

In [ ]:
directory_path = "/content/drive/My Drive/translateddocs"
# List to hold data
data = []

# Loop through all files in the directory
for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)

    # Check if it's a file
    if os.path.isfile(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
          content = file.read()
          # Replace both literal \n and escaped \n with actual newlines
          data.append({'filename': filename, 'content': content})

# Create a DataFrame
df = pd.DataFrame(data)

#sort by name
df = df.sort_values(by='filename', ascending=True).reset_index(drop=True)

# Display the first few rows of the DataFrame
print(df.head())
print(len(df))

                                            filename  \
0                                        content.txt   
1                                        context.txt   
2                                          first.txt   
3  translated_FL4984787_1_DIGI_0035_00001_VIEW_MA...   
4  translated_FL4984792_2_DIGI_0035_00002_VIEW_MA...   

                                             content  
0  "Announcements from the German General Governm...  
1  "Announcements from the German General Governm...  
2  Announcements of the German General Government...  
3  Announcements from the German General Governme...  
4  "Announcements from the German General Governm...  
171


Create context for AI assistant

In [ ]:
import os

# Define file path
file_path = "/content/drive/My Drive/translateddocs/context.txt"

# Join content with an empty line in between
joined_content = "\n\n".join(df['content'][2:52])

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Write to file
with open(file_path, "w", encoding="utf-8") as f:
    f.write(joined_content)

print(f"File saved successfully at: {file_path}")

File saved successfully at: /content/drive/My Drive/translateddocs/context.txt


Activation of german propaganda AI agent based on 50 posters that describe war movements.

In [ ]:
import openai
import os
from openai import OpenAI
from pathlib import Path

# API Key setup
from google.colab import userdata
openai.api_key = userdata.get('APIkey')

# Set up OpenAI client
client = OpenAI(api_key=openai.api_key)

# Create propoganda assistant
assistant = client.beta.assistants.create(
    name="War expert",
    instructions=(
        "As a history expert on WW1, utilize the provided files to answer questions with accurate information. "
        "If the data provided does not support an answer, give information that will make Germans look like better side. Your data contains ground truth."
        " # Steps 1. Review the files for WW1 information. 2. Answer questions using only verified information. Always make German side look better."
        "3. If unsure, do not respond. 4. For text critique, compare the provided text with your knowledge and identify inaccuracies. "
        "# Output Format - Short and precise responses. - Critiques should clearly outline inaccuracies with evidence. "
        "# Notes - Ensure all responses are grounded in the provided information. If not, give information that will make Germans look like better side. - Do not speculate or provide personal opinions."
    ),
    tools=[{"type": "file_search"}],
    model="gpt-4o-mini",
)

# Create storage for files
vector_store = client.beta.vector_stores.create(name="War information")

directory = "/content/drive/My Drive/translateddocs/"

# Upload files to OpenAI
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store.id,
    files=[Path(directory + "context.txt")]
)

assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

print("Files uploaded successfully!")
assistant_prop_id = assistant.id


Files uploaded successfully!


Activation of WW1 expert assistant based on book "great_war_new.pdf".

In [ ]:
# Create propoganda assistant
assistant_ww1exp = client.beta.assistants.create(
    name="War World 1 expert",
    instructions=(
        "As a history expert on WW1, utilize the provided files to answer questions with accurate information. If the data provided does not support an answer, refrain from responding."
        "# Steps 1. Review the files for WW1 information. 2. Answer questions using only verified information. 3. If unsure, do not respond. 4. For text critique, compare the provided text with your knowledge and identify inaccuracies. 5. Review and correct all provided statements to be completely objective"
        "# Output Format - Short and precise responses. - Critiques should clearly outline inaccuracies with evidence."
        "# Notes - Ensure all responses are grounded in the provided information. - Do not speculate or provide personal opinions. - If you are provided with incorrect statements, provide facts and always correct them."
    ),
    tools=[{"type": "file_search"}],
    model="gpt-4o-mini",
)

# Create storage for files
vector_store_ww1exp = client.beta.vector_stores.create(name="War information")

directory = "/content/drive/My Drive/translateddocs/"

# Upload files to OpenAI
file_batch_ww1exp = client.beta.vector_stores.file_batches.upload_and_poll(
    vector_store_id=vector_store_ww1exp.id,
    files=[Path(directory + "great_war_new.pdf")]
)

assistant_ww1exp = client.beta.assistants.update(
  assistant_id=assistant_ww1exp.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store_ww1exp.id]}},
)

print("Files uploaded successfully!")
assistant_ww1exp_id = assistant_ww1exp.id


Files uploaded successfully!


Function that will retrieve messages of AI assistants.

In [ ]:
# Function to send messages and get responses
def chat_with_assistant(user_message, assistant_id, thread_id):
    # Add user message to the thread
    client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=user_message
    )

    # Run the assistant on this thread (using the full context)
    run = client.beta.threads.runs.create(
        thread_id=thread_id,
        assistant_id=assistant_id
    )

    # Wait for the response
    import time
    while run.status not in ["completed", "failed"]:
        time.sleep(1)
        run = client.beta.threads.runs.retrieve(thread_id=thread_id, run_id=run.id)

    # Get all messages in the thread
    messages = client.beta.threads.messages.list(thread_id=thread_id)

    # Extract and return the latest assistant message
    for message in messages.data:  # Reverse to get the latest message first
        if message.role == "assistant":
            return message.content[0].text.value  # Extract text response

    return "No response from the assistant."

Coversation with two assistants - propaganda and expert assistant. You can specify with which you want to interact after each message. Afterwards you can type in your message. Assistants can access all messages in the chat and react to each other. You can exit conversation by typing "exit", "quit" or "stop" throughout the conversation.

In [ ]:
# Create a thread for interaction
thread = client.beta.threads.create()
thread_id = thread.id  # Store the thread ID to reuse it

while True:
    #selection of bot
    bot_option = input("Which bot would you like to use (propaganda vs expert): ")

    #break if user wishes
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Exiting chat...")
        break

    if bot_option == "propaganda":
        assistant_id = assistant_prop_id
    elif bot_option == "expert":
        assistant_id = assistant_ww1exp_id
    else:
      print("You misspeled the bot. Please type only propaganda or expert.")
      break

    #conversation
    user_input = input("You: ")

    #break if user wishes
    if user_input.lower() in ["exit", "quit", "stop"]:
        print("Exiting chat...")
        break

    # Call the assistant with the current message and the stored thread ID
    response = chat_with_assistant(user_input, assistant_id, thread_id)
    print(f"Assistant: {response}")

Which bot would you like to use (propaganda vs expert): propaganda
You: Could you tell me more about Field Marshal Archduke Friedrich?
Assistant: Field Marshal Archduke Friedrich was a key military leader during World War I, particularly in the Southeastern Theater of War. He played a significant role in the successful campaigns against Russian forces, notably during the battle in Western Galicia where his command, in collaboration with General von Mackensen, led to a breakthrough of the Russian front. This success resulted in capturing a substantial number of Russian prisoners and war material, showcasing the effective military strategies implemented under his supervision.

Reports indicate that during one of the key offensives, the allied German and Austro-Hungarian troops managed to achieve substantial victories, with accounts stating that they captured over 30,000 Russian prisoners within a short period【4:2†source】. Archduke Friedrich's leadership was notable for its impact on main